In [1]:
!pip install -qU crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.237.0 requires protobuf<5.0,>=3.12, but you have protobuf 5.29.2 which is incompatible.


In [30]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [31]:
import os
from utils import get_bedrock_client, get_haiku_config, get_sonnet_config
from langchain_community.chat_models import BedrockChat  # Changed this import
from crewai import Agent, Task, Crew

In [32]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


bedrock_client = get_bedrock_client()
sonnet_config = get_sonnet_config()  # Using Sonnet config

# Create BedrockChat instance for CrewAI
llm = BedrockChat(
    client=bedrock_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # Updated model ID
    streaming=True,
    model_kwargs={
        "temperature": sonnet_config["temperature"],
        "top_p": sonnet_config["top_p"],
        "max_tokens": sonnet_config["max_tokens"],
        "anthropic_version": "bedrock-2023-05-31"
    }
)

# Set model name in environment if needed
os.environ["MODEL_NAME"] = "anthropic.claude-3-sonnet-20240229-v1:0"

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [33]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
)

### Agent: Writer

In [20]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
     llm=llm,
    max_iter=5,
    memory=True,
)

### Agent: Editor

In [34]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [36]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [37]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [38]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [39]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-01-01 10:50:15,970 - 140055258498880 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [40]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I will create a comprehensive content plan document covering the key aspects of Artificial Intelligence, including the latest trends, major players, target audience analysis, content outline with an introduction, main points, call to action, SEO keywords, and relevant data sources. This will serve as a solid foundation for the Content Writer to craft an engaging and informative article on this topic.

Final Answer:

Content Plan: Artificial Intelligence

Target Audience Analysis:
- Tech enthusiasts interested in the latest advancements in AI
- B

In [27]:
from IPython.display import Markdown
Markdown(result)

# Exploring the Wonders of Artificial Intelligence

## Introduction

In today's rapidly evolving technological landscape, artificial intelligence (AI) has emerged as a groundbreaking field capturing the world's imagination. From virtual assistants that simplify daily tasks to self-driving vehicles promising a transportation revolution, AI is permeating every aspect of our lives. As we delve deeper into this fascinating realm, understanding the fundamentals, applications, and implications of this technology becomes increasingly crucial.

AI encompasses various subfields, including machine learning, deep learning, and neural networks. At its core, AI aims to develop intelligent systems capable of perceiving, learning, reasoning, and acting in ways that mimic or even surpass human cognitive abilities. The rapid advancements in AI have been driven by the exponential growth of computing power, the availability of vast data, and the development of sophisticated algorithms.

## The Current State of AI

Leading tech companies like Google, Microsoft, and OpenAI have spearheaded the race to unlock AI's full potential through substantial investments in cutting-edge research and development. These efforts have yielded remarkable breakthroughs, such as ChatGPT, a conversational AI model that can engage in human-like dialogue, and DALL-E, a system that generates realistic images from textual descriptions.

AI's applications span a wide range of domains, from image recognition and natural language processing to predictive analytics and decision-making systems. The healthcare industry, for instance, is leveraging AI to improve disease diagnosis, drug discovery, and personalized treatment plans. In finance, AI algorithms are employed for fraud detection, risk assessment, and portfolio optimization.

## AI in Everyday Life

Beyond specialized industries, AI has already woven itself into our everyday lives. Virtual assistants like Siri, Alexa, and Google Assistant have become indispensable companions, helping us manage schedules, control smart home devices, and access information with simple voice commands. Recommendation systems powered by AI algorithms curate personalized content for us on streaming platforms and e-commerce websites, enhancing our user experience.

As AI continues to evolve, its impact is poised to transform various sectors. Autonomous vehicles, once a distant dream, are now inching closer to reality, promising safer and more efficient transportation. In manufacturing, AI-powered robotics are streamlining production processes, increasing efficiency, and reducing human error.

## The Future of AI

While the current achievements of AI are remarkable, many experts believe we are merely scratching the surface of its true potential. The pursuit of artificial general intelligence (AGI), a system capable of matching or surpassing human intelligence across a broad range of tasks, remains an ambitious goal. Some even speculate about the possibility of superintelligence, an AI that vastly exceeds human cognitive capabilities.

As AI continues to advance, it presents both opportunities and challenges. On one hand, AI could help address global issues such as climate change, disease prevention, and resource scarcity through its ability to process vast amounts of data and identify patterns and solutions. On the other hand, the automation of tasks and jobs raises concerns about job displacement and the need for workforce retraining.

## Ethical Considerations

With great power comes great responsibility, and the development of AI raises critical ethical considerations. Privacy and data security are paramount concerns, as AI systems often rely on vast amounts of personal data for training and decision-making. Ensuring the fairness and accountability of AI systems is crucial to prevent biases and discrimination.

Transparency in AI decision-making processes is another area of focus, as the complexity of these systems can make it challenging to understand how they arrive at specific outcomes. Regulatory frameworks and guidelines are being developed to address these ethical challenges and ensure the responsible development and deployment of AI technologies.

## Staying Informed

As we stand at the precipice of an AI-driven future, it is essential for all of us to stay informed about the latest developments and their implications. Engage with reputable sources, attend workshops or online courses, and actively participate in discussions surrounding AI's impact on society.

Share your thoughts or experiences with AI in the comments below. Together, we can navigate this exciting and transformative era, harnessing the power of AI while upholding ethical principles and safeguarding the well-being of humanity.